<a href="https://colab.research.google.com/github/kai0200/MaJiangGame/blob/main/%E5%9B%9B%E5%B7%9D%E9%BA%BB%E5%B0%86%E9%80%89%E7%89%8C%E7%AD%96%E7%95%A5%E5%87%BD%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
四川麻将选牌策略函数

功能：
    根据手牌、摸牌和已打出的牌，选择一张最优的牌打出。

输入：
    hand (字典): 一个表示手牌的字典。
        - 键为花色: '筒', '条', '万'
        - 值为一个长度为9的列表，索引 0 对应牌点 1，索引 1 对应牌点 2，以此类推。
          列表中的值为该牌点牌的数量 (0-4)。
        例如:
        {
            '筒': [0, 2, 0, 1, 0, 0, 0, 0, 0],
            '条': [0, 0, 0, 0, 2, 2, 2, 0, 0],
            '万': [2, 0, 0, 0, 0, 0, 0, 0, 2]
        }
    drawn_card (元组): 摸到的牌，格式为 (花色, 点数)，例如 ('筒', 2)。
    discarded_cards (列表): 已经打出的牌的列表。
        - 列表中的每个元素也是一个元组，格式为 (花色, 点数)，
          例如 [('筒', 1), ('条', 3), ('万', 5)]。

输出：
    元组: 打出的牌，格式为 (花色, 点数)，例如 ('筒', 1)。
    如果无法决策 (不应该发生)，返回 None。

选牌策略：
    1.  评估手牌胡牌的概率和番数，计算期望收益（赔率 * 概率）。
    2.  逐级判断，从期望收益最高的牌开始考虑，到最低的牌。
    3.  对于每一张考虑打出的牌，判断打出后是否会减少胡牌的可能性：
        a.  判断已打出的牌中，是否已经有较多数量的这张牌的搭子牌。
        b.  如果打出会减少胡牌的可能性，则考虑下一张牌。
    4.  如果所有牌都会减少胡牌的可能性，则选择一张相对最没有用的牌打出。
        - 优先打出孤张（没有搭子的牌）。
        - 如果有多个孤张，打出最不可能成搭的牌（例如，已经打出较多同花色邻牌的牌）。
"""

import random

def choose_discard(hand, drawn_card, discarded_cards):
    """
    选择一张要打出的牌。

    Args:
        hand (dict): 表示手牌的字典。
        drawn_card (tuple): 摸到的牌。
        discarded_cards (list): 已经打出的牌的列表。

    Returns:
        tuple: 打出的牌。
    """

    # 1. 将摸到的牌加入手牌
    hand = add_card_to_hand(hand, drawn_card)

    # 2. 评估手牌的胡牌概率和番数
    card_values = evaluate_hand(hand, discarded_cards) # 得到每张牌的价值

    # 3. 按照价值从高到低排序
    sorted_cards = sorted(card_values.items(), key=lambda item: item[1]['expected_value'], reverse=True)

    # 4. 逐张考虑打出
    for card_tuple, card_info in sorted_cards:
        if not will_reduce_winning_chance(hand, card_tuple, discarded_cards):
            return card_tuple # 找到一张合适的

    # 5. 如果所有牌都会降低胡牌几率，选择相对最没用的牌
    return choose_least_valuable_card(hand, discarded_cards)

def add_card_to_hand(hand, card):
    """
    将一张牌加入手牌。

    Args:
        hand (dict): 表示手牌的字典。
        card (tuple): 要加入的牌。

    Returns:
        dict: 更新后的手牌。
    """
    new_hand = copy_hand(hand)
    new_hand[card[0]][card[1] - 1] += 1
    return new_hand

def remove_card_from_hand(hand, card):
    """
    从手牌中移除一张牌
    Args:
        hand (dict): 手牌
        card (tuple): 要移除的牌
    Returns:
        dict: 移除后的手牌
    """
    new_hand = copy_hand(hand)
    new_hand[card[0]][card[1] - 1] -= 1
    return new_hand

def copy_hand(hand):
    """
    复制手牌字典，避免修改原始数据。

    Args:
        hand (dict): 要复制的手牌字典。

    Returns:
        dict: 复制后的手牌字典。
    """
    new_hand = {}
    for suit in hand:
        new_hand[suit] = list(hand[suit])  # 复制内部的列表
    return new_hand

def evaluate_hand(hand, discarded_cards):
    """
    评估手牌中每张牌的价值，包括胡牌概率和番数。

    Args:
        hand (dict): 表示手牌的字典。
        discarded_cards (list): 已经打出的牌的列表。

    Returns:
        dict: 一个字典，键为牌的元组 (花色, 点数)，值为一个字典，包含 'winning_chance' (胡牌概率) 和 'expected_value' (期望收益)。
    """
    card_values = {}
    for suit in hand:
        for i, count in enumerate(hand[suit]):
            if count > 0:
                card = (suit, i + 1)
                # 临时移除这张牌，计算移除后的手牌价值
                temp_hand = remove_card_from_hand(hand, card)
                winning_chance = calculate_winning_chance(temp_hand, discarded_cards) # 计算概率
                # 这里简化了番数的计算，实际游戏中需要更复杂的逻辑
                expected_value = winning_chance * 1  # 假设每种胡牌都是1番，这里需要替换成实际的番数计算
                card_values[card] = {
                    'winning_chance': winning_chance,
                    'expected_value': expected_value,
                }
    return card_values

def calculate_winning_chance(hand, discarded_cards):
    """
    计算当前手牌的胡牌概率的简单估计。  这里可以根据实际情况使用更复杂的算法。

    Args:
        hand (dict): 手牌
        discarded_cards (list): 已打出的牌

    Returns:
        float: 胡牌概率 (0-1)
    """
    remaining_cards = 0
    needed_cards = 0

    # 1. 统计剩余牌数
    for suit in hand:
        for count in hand[suit]:
            remaining_cards += (4-count) # 简化计算，认为剩余所有牌都能摸到

    # 2. 统计需要的牌数 (听牌张数)
    possible_hands = get_possible_winning_hands(hand) # 得到所有可能和的牌

    for possible_hand in possible_hands:
        suit = possible_hand[0]
        index = possible_hand[1] - 1
        count_discarded = 0
        for card in discarded_cards:
            if card[0] == suit and card[1] == possible_hand[1]:
                count_discarded += 1
        if hand[suit][index] < 4:
            needed_cards += (4 - hand[suit][index] - count_discarded)

    if remaining_cards == 0:
        return 0
    return min(1.0, needed_cards / remaining_cards) # 简化计算，实际概率计算需要考虑更多因素

def get_possible_winning_hands(hand):
    """
    获取所有可能胡的牌
    Args:
        hand (dict): 手牌
    Returns:
        list: 所有可能胡的牌的列表，格式为 (花色, 点数)
    """
    possible_hands = []
    for suit in hand:
        for i in range(9):
            if hand[suit][i] == 0:
                possible_hands.append((suit, i + 1))
    return possible_hands

def will_reduce_winning_chance(hand, card_to_discard, discarded_cards):
    """
    判断打出某张牌是否会减少胡牌的可能性。

    Args:
        hand (dict): 表示手牌的字典。
        card_to_discard (tuple): 要打出的牌。
        discarded_cards (list): 已经打出的牌的列表。

    Returns:
        bool: True 表示会减少，False 表示不会减少。
    """
    temp_hand = remove_card_from_hand(hand, card_to_discard) # 移除
    original_winning_chance = calculate_winning_chance(hand, discarded_cards)
    new_winning_chance = calculate_winning_chance(temp_hand, discarded_cards)
    return new_winning_chance < original_winning_chance

def choose_least_valuable_card(hand, discarded_cards):
    """
    选择一张相对来说最没有用的牌打出。

    Args:
        hand (dict): 表示手牌的字典。
        discarded_cards (list): 已经打出的牌的列表。

    Returns:
        tuple: 要打出的牌。
    """
    least_valuable_card = None
    min_potential_value = float('inf')

    for suit in hand:
        for i, count in enumerate(hand[suit]):
            if count > 0:
                card = (suit, i + 1)
                potential_value = calculate_card_potential_value(hand, card, discarded_cards) # 计算单张牌的价值
                if potential_value < min_potential_value:
                    min_potential_value = potential_value
                    least_valuable_card = card
    return least_valuable_card

def calculate_card_potential_value(hand, card, discarded_cards):
    """
    计算一张牌的潜在价值，即这张牌在未来组成搭子的可能性。

    Args:
        hand (dict): 手牌
        card (tuple): 要评估的牌
        discarded_cards (list): 已打出的牌

    Returns:
        int: 牌的潜在价值，越低表示越没用
    """
    suit, rank = card
    potential_value = 0

    # 1. 考虑是否是孤张
    if hand[suit][rank - 1] == 1:
        potential_value += 1

    # 2. 考虑邻牌被打出的情况
    for i in range(max(0, rank - 2), min(9, rank + 1)):
        if i != rank -1 :
            discard_count = 0
            for discarded_card in discarded_cards:
                if discarded_card[0] == suit and discarded_card[1] == i + 1:
                    discard_count += 1
            potential_value += discard_count
    return potential_value

def print_hand(hand):
    """
    格式化打印手牌

    Args:
        hand (dict): 手牌
    """
    suit_order = ['筒', '条', '万']
    for suit in suit_order:
        print(f"{suit}: ", end="")
        for i, count in enumerate(hand[suit]):
            if count > 0:
                print(f"{i+1}*{count} ", end="")
        print()

if __name__ == "__main__":
    # 示例
    hand = {
        '筒': [0, 2, 0, 1, 0, 0, 0, 0, 0],
        '条': [0, 1, 0, 0, 2, 2, 1, 0, 0],
        '万': [2, 0, 0, 0, 1, 0, 0, 0, 1]
    }
    drawn_card = ('筒', 2)
    discarded_cards = [('筒', 1), ('条', 3), ('万', 5), ('筒', 3), ('筒', 1), ('条', 4)]

    print("初始手牌：")
    print_hand(hand)
    print(f"摸到的牌：{drawn_card}")
    print(f"已打出的牌：{discarded_cards}")

    discard_card = choose_discard(hand, drawn_card, discarded_cards)
    print(f"应该打出的牌：{discard_card}")

    print("\n打出后的手牌：")
    hand = remove_card_from_hand(hand, discard_card)
    print_hand(hand)